## Dealing with C structs in Python

Python has a builtin library for reading byte streams into and out of Python objects.

In [ ]:
import struct

In [ ]:
data = struct.pack("id5s", 1, 2.2, "THREE")
print data
print map(ord, data)

In [ ]:
struct.unpack("id5s", data)

| Format | C Type | Python type | Standard size |
|:-------|:-------|:------------|:--------------|
| x | pad byte | no value | 1 |
| c | `char` | string of length 1 | 1 |
| b | `signed char` | `int` | 1 |
| B | `unsigned char` | `int` | 1 |
| ? | `_Bool` | `bool` | 1 |
| h | `short` | `int` | 2 |
| H | `unsigned short` | `int` | 2 |
| i | `int` | `int` | 4 |
| I | `unsigned int` | `int` | 4 |
| l | `long` | `int` | 4 |
| L | `unsigned long` | `int` | 4 |
| q | `long long` | `int` | 8 |
| Q | `unsigned long long` | `int` | 8 |
| f | `float` | `float` | 4 |
| d | `double` | `float` | 8 |
| s | `char[]` | `string` | |
| p | `char[]` | `string` | |
| P | `void *` | `int` | |


While this is fine for occasional things, like file headers or a couple of network packets, it's not an efficient way of viewing a large number of structs, such as an array of structs.

For that, there's Numpy (naturally).

Numpy has a [record array](https://docs.scipy.org/doc/numpy/user/basics.rec.html) type that is formed by passing it a multi-valued dtype.

In [ ]:
import numpy
array = numpy.empty(100, dtype=[("one", numpy.int32),
                                ("two", numpy.float64),
                                ("three", "|S5")])

In [ ]:
array["one"][0] = 1
array["two"][0] = 2.2
array["three"][0] = "THREE"

In [ ]:
array[0]

In [ ]:
array.view("S1")[: 4 + 8 + 5]

Note that these bytes are packed more tightly than what `struct.pack` gave us. `struct.pack` is following C's rules for data padding. We can get the same behavior by explicitly asking for alignment in the dtype.

In [ ]:
array = numpy.empty(100, dtype=numpy.dtype(
    [("one", numpy.int32),
     ("two", numpy.float64),
     ("three", "|S5")], align=True))

array["one"][0] = 1
array["two"][0] = 2.2
array["three"][0] = "THREE"

array.view("S1")[:len(data)]

In fact, the offset of each field in the record array can be individually set.